In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
testCSV = pd.read_csv('test.csv')

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
bow = CountVectorizer(max_features=10000000,
                      lowercase=True,
                      ngram_range=(1,1),
                      analyzer = "word").fit(testCSV['Lyrics'].values.astype(str))
len(bow.vocabulary_)

11760

In [5]:
lyrics_bow = bow.transform(testCSV['Lyrics'].values.astype(str))
print('Shape of Sparse Matrix: ', lyrics_bow.shape)
lyrics_bow.nnz

Shape of Sparse Matrix:  (2580, 11760)


86667

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer().fit(lyrics_bow)
lyrics_tfidf = tfidf_transformer.transform(lyrics_bow)

In [7]:
#f_X = lyrics_tfidf
#df_X.headl
lyrics_tfidf
df_X = pd.SparseDataFrame(lyrics_tfidf)

KeyboardInterrupt: 

In [8]:
lyrics_tfidf

<2580x11760 sparse matrix of type '<class 'numpy.float64'>'
	with 86667 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()      #.fit(lyrics_tfidf, testCSV['hitOrNot'])

from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(lyrics_tfidf, testCSV['hitOrNot'], test_size=0.2,random_state = 15)
model.fit(X_train, Y_train)
predictions = model.predict(X_test)

In [20]:
X_test

<516x11760 sparse matrix of type '<class 'numpy.float64'>'
	with 17555 stored elements in Compressed Sparse Row format>

In [21]:
from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))

             precision    recall  f1-score   support

          0       0.59      1.00      0.74       301
          1       1.00      0.01      0.02       215

avg / total       0.76      0.59      0.44       516



In [22]:
from sklearn.linear_model import LogisticRegression

In [23]:
modelLog = LogisticRegression()

modelLog.fit(X_train, Y_train)
predictions = modelLog.predict(X_test)

from sklearn.metrics import classification_report
print(classification_report(Y_test, predictions))

             precision    recall  f1-score   support

          0       0.59      0.88      0.70       301
          1       0.44      0.13      0.20       215

avg / total       0.52      0.57      0.49       516



# Testing model

In [24]:
    print(model.predict(lyrics_tfidf)[7])
    print(testCSV.hitOrNot[7])

0
1


In [25]:
testCSV.head()

,Unnamed: 0,SongNumber,SongID,AlbumID,AlbumName,ArtistID,ArtistName,Duration,KeySignature,KeySignatureConfidence,Tempo,TimeSignature,TimeSignatureConfidence,Title,Year,song_hotttnesss,Lyrics,hitOrNot,Rating
0,0,1,SOQUGNH12A6D4FBB0E,64577,Genesis And Jo,ARLQ5EV1187FB4491D,Don Francisco,311.22240,0,0.665,68.537,1,0.000,Overture (Genesis & Job),0,0.000000,instrumental,0,1
1,1,2,SOPHMRL12AB017E87,401052,Not Without A Fight,ARH9HS41187FB5BC6F,New Found Glory,208.27383,4,0.662,192.070,4,0.937,"bDont Let Her Pull You Down""""",2009,0.663194,time rain parade watch hope explode landmines ...,1,4
2,2,3,SOQENPJ12AB0180997,532011,Listening Tree,AR8U8G41187B9A833,Tim Exile,369.57995,1,0.000,110.000,4,0.662,Fortress,2009,0.530026,who coming stair shouting in coming dying like...,1,3
3,3,4,SOOCZWD12AB0185147,503069,Desolation Angels,ARD8JVH1187FB4DA04,Bad Company,211.14730,0,0.614,194.283,7,0.470,Crazy Circles,1979,0.552548,life like merry go round painted horse riding ...,1,3
4,4,5,SOYMDLM12A6D4FB10,59445,Mozart: Academy in Concert,ARBH0MS1187FB36311,Sir Neville Marriner,296.80281,7,0.651,61.634,5,0.527,Two Elegiac Melodies Op. 34: 2. The Last Spring,0,0.588922,instrumental,1,3


In [26]:
#global user_lyrics_tfidf
userLyrics = "time rain parade watch hope explode landmines ask help people turn away your living far away truth your believing lie surprise sleep night drowning prize made feel welcome welcome make forget poison running vein may say love love lie mess you made dont let dont let pull dont let dont let pull dont let"

In [27]:
def basicLyricsProcessing(userLyrics):
    #Everything in lowercase
    lowerCase = lambda x: " ".join(x.lower() for x in str(x).split())
    userLyrics = lowerCase(userLyrics)

    #Removing punctuation that does not add meaning to the song
    userLyrics = userLyrics.replace('[^\w\s]','')
    
    #Removing of stop words
    from nltk.corpus import stopwords

    stop = stopwords.words('english')
    removeStopWords = lambda x: " ".join(x for x in str(x).split() if x not in stop)
    userLyrics = removeStopWords(userLyrics)
    
    #Correction of Spelling mistakes
    from textblob import TextBlob
    spellingMistake = lambda x: str(TextBlob(x).correct())
    userLyrics = spellingMistake(userLyrics)
    
    #Lemmatization is basically converting a word into its root word. It is preferred over Stemming.
    from textblob import Word
    lemmatize = lambda x: " ".join([Word(word).lemmatize() for word in x.split()])
    userLyrics = lemmatize(userLyrics)
    
    #CountVectorization of user lyrics
    from sklearn.feature_extraction.text import CountVectorizer
    user_bow = CountVectorizer(max_features=100000,
                          lowercase=True,
                          ngram_range=(1,1),
                          analyzer = "word").fit([userLyrics])
    
    #Bag of Words conversion of user lyrics
    user_lyrics_bow = bow.transform([userLyrics])
    
    #Tf-idf transforming of user lyrics
    from sklearn.feature_extraction.text import TfidfTransformer
    user_tfidf_transformer = TfidfTransformer().fit(user_lyrics_bow)
    user_lyrics_tfidf = user_tfidf_transformer.transform(user_lyrics_bow)
    return user_lyrics_tfidf

In [31]:
user_tfidf = basicLyricsProcessing(userLyrics)

LookupError: 
**********************************************************************
  Resource 'corpora/stopwords' not found.  Please use the NLTK
  Downloader to obtain the resource:  >>> nltk.download()
  Searched in:
    - 'C:\\Users\\LAKSHAY/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\LAKSHAY\\AppData\\Local\\Enthought\\Canopy\\edm\\envs\\User\\nltk_data'
    - 'C:\\Users\\LAKSHAY\\AppData\\Local\\Enthought\\Canopy\\edm\\envs\\User\\lib\\nltk_data'
    - 'C:\\Users\\LAKSHAY\\AppData\\Roaming\\nltk_data'
**********************************************************************